In [ ]:
import pandas as pd
from transformers import AutoTokenizer
import torch
import numpy as np

In [2]:
#import data
training_df = pd.read_csv('data/train.csv')
testing_df = pd.read_csv('data/test.csv')
training_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [4]:
#tokenize the training data
text = training_df['full_text'].to_list()
encoded_input = tokenizer(text, padding = True, truncation=True, return_tensors = 'pt')

#build a vocabulary vector
tokens = encoded_input['input_ids']
global max_num
max_num = 0
for token in tokens:
    max_temp = int(torch.max(token))
    if max_temp > max_num:
        max_num = max_temp
    else:
        continue


#convert the sequencial token to vector token
train_token_list = []
for token in tokens:
    init_vector = np.zeros(max_num+1)
    for position_index in token:
        i = int(position_index)
        if i > max_num:
            continue
        else:
            init_vector[i] = init_vector[i]+1
    train_token_list.append(init_vector)

In [5]:
#tokenize the testing data
text = testing_df['full_text'].to_list()
encoded_input = tokenizer(text, padding = True, truncation=True, return_tensors = 'pt')

tokens = encoded_input['input_ids']

#convert the sequencial token to vector token
test_token_list = []
for token in tokens:
    init_vector = np.zeros(max_num+1)
    for position_index in token:
        i = int(position_index)
        if i > max_num:
            continue
        else:
            init_vector[i] = init_vector[i]+1
    test_token_list.append(init_vector) 

In [6]:
#get all the scores in the training samples
vocabulary_scores = training_df.vocabulary

#get a list of all possible scores
vocabulary_scores_range = set(vocabulary_scores)

total_case_num = len(vocabulary_scores) #get total training sample size

#get the number of xi in each score class
x_prob_dict = {}

#get the number of yi in each score class
total_case_dict = {}

for score in vocabulary_scores_range:
    total_case_dict[score] = 0
    x_prob_dict[score] = np.ones(len(train_token_list[0])-1)

for i in range(0, len(vocabulary_scores)):
    total_case_dict[vocabulary_scores[i]] += 1
    sample = train_token_list[i]
    for j in range(1,len(sample)):
        if sample[j] > 0:
            x_prob_dict[vocabulary_scores[i]][j-1] += 1

In [76]:
x_prob_df = pd.DataFrame(x_prob_dict)
x_prob_df = x_prob_df[x_prob_df.columns.sort_values()]

In [77]:
x_prob_df

,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
28116,1.0,1.0,14.0,30.0,93.0,51.0,25.0,1.0,3.0
28117,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0
28118,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
28119,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

X = np.array(train_token_list)
y = vocabulary_scores.to_numpy()*2

cut_point = int(np.floor(0.8*X.shape[0]))
X_train = X[:cut_point,:]
y_train = y[:cut_point]
X_test = X[cut_point:,:]
y_test = y[cut_point:]

clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [60]:
y_pred = clf.predict(X_test)/2

In [62]:
count = 0
total_num = len(y_pred)
for i in range(0,len(y_pred)):
    if y_test[i]/2 == y_pred[i]:
        count += 1
print('The correct rate is', count/total_num)

The correct rate is 0.3116219667943806


3911